In [3]:
import pandas as pd
from top2vec import Top2Vec
from tqdm import tqdm
import nltk
stop=nltk.corpus.stopwords.words('english')
import spacy
nlp=spacy.load('en_core_web_sm')
from nltk import word_tokenize

/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packag

In [5]:
first=pd.read_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/kkk-first-label.csv')
between=pd.read_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/kkk-between-label.csv')
second=pd.read_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/kkk-second-label.csv')

In [4]:
def lemmatization(dataframe:pd.DataFrame()):
    dataframe['article']=dataframe['article'].str.lower()
    dataframe['stopword']=dataframe['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    dataframe['punct']=dataframe['stopword'].str.replace('[^\w\s]','')
    dataframe['lemma']=dataframe['punct'].apply(lambda row: ' '.join([w.lemma_ for w in nlp(row)]))
    dataframe['token']=dataframe['lemma'].apply(word_tokenize)
    return dataframe

In [6]:
african_second_lemma=lemmatization(second)
#second: 25m 30s

In [7]:
model=Top2Vec(documents=african_second_lemma['lemma'].tolist(), speed='learn', workers=4)
#second: 12m 21s

2023-10-14 17:16:56,894 - top2vec - INFO - Pre-processing documents for training
2023-10-14 17:17:05,629 - top2vec - INFO - Creating joint document/word embedding
2023-10-14 17:28:33,023 - top2vec - INFO - Creating lower dimension embedding of documents
2023-10-14 17:28:46,428 - top2vec - INFO - Finding dense areas of documents
2023-10-14 17:28:47,770 - top2vec - INFO - Finding topics


In [27]:
model.save("/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/kkk-second-model")


Load Top2vec model

In [30]:
model = Top2Vec.load("/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/kkk-second-model")

In [9]:
topic_sizes, topic_nums = model.get_topic_sizes()

In [10]:
print(len(topic_sizes), len(topic_nums))

281 281


In [11]:
id_dic={}
topic_id={}
for element in zip(topic_nums, topic_sizes):
    documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=element[0], num_docs=element[1])
    for score, id in zip(document_scores, document_ids):
        id_dic[id]=score
        topic_id[id]=element[0]

In [13]:
topic_words, word_scores, topic_scores = model.get_topics(len(topic_sizes))

In [14]:
topic_words

array([['bandit', 'robber', 'robbery', ..., 'ransack', 'steal',
        'messenger'],
       ['creed', 'americanism', 'racial', ..., 'welfare', 'ligious',
        'country'],
       ['stare', 'softly', 'you', ..., 'dear', 'exclaim', 'door'],
       ...,
       ['goose', 'flog', 'fiogge', ..., 'lehi', 'morehouse', 'beaumont'],
       ['sargent', 'gail', 'investment', ..., 'profit', 'worthless',
        'disapproval'],
       ['aileen', 'charming', 'synopsis', ..., 'cn', 'you', 'curiosity']],
      dtype='<U15')

In [15]:
df_words=pd.DataFrame(topic_words).transpose()
df_words.columns=topic_nums
df_words.columns = df_words.columns.astype(str)

In [16]:
df_words.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/second-topic-words.csv')

In [18]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["wizard"], num_topics=5)
topic_words

[array(['sore', 'canker', 'cold', 'loosen', 'soothe', 'earache', 'heal',
        'toothache', 'gargle', 'constipation', 'neck', 'chest', 'troubled',
        'rub', 'tired', 'quick', 'generous', 'sprain', 'often', 'bottle',
        'relief', 'sick', 'headache', 'medicine', 'hamiin', 'druggist',
        'liver', 'bruise', 'pill', 'aration', 'shelf', 'throat', 'prep',
        'ache', 'pink', 'simple', 'pleasant', 'guarantee', 'burn',
        'reliable', 'satisfied', 'hamlin', 'stiff', 'dependable', 'stings',
        'bite', 'occur', 'deep', 'constipate', 'healing'], dtype='<U15'),
 array(['soothe', 'penetrate', 'heal', 'constipation', 'reliable',
        'generous', 'troubled', 'sprain', 'sick', 'stings', 'sting',
        'druggist', 'bite', 'sore', 'headache', 'soreness', 'ache',
        'stiff', 'medicine', 'bruise', 'liver', 'hamiin', 'canker',
        'bottle', 'pill', 'sooth', 'earache', 'limber', 'absolutely',
        'pleasant', 'antiseptic', 'pink', 'toothache', 'guarantee',
     

In [20]:
print(topic_nums)
print(topic_scores)

[ 72 221  92  21 128]
[0.26426785 0.25976787 0.25188837 0.23514197 0.21826114]


In [23]:
wizard_relevant_index=[]
wizard_relevant_score=[]
wizard_relevant_lemma=[]
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=21, num_docs=411)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    wizard_relevant_index.append(doc_id)
    wizard_relevant_score.append(score)
    wizard_relevant_lemma.append(doc)

In [36]:
pd.DataFrame.from_dict({'index':wizard_relevant_index, 'score':wizard_relevant_score, 'lemma':wizard_relevant_lemma}).to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-wizard-relevant-df.csv', index=False)

In [32]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["lynch"], num_docs=second.shape[0])
lynch_df=pd.DataFrame({'doc':documents, 'id':document_ids, 'score':document_scores})
lynch_df.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-lynch-topic.csv')

In [33]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["riot"], num_docs=second.shape[0])
riot_df=pd.DataFrame({'doc':documents, 'id':document_ids, 'score':document_scores})
riot_df.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-riot-topic.csv')

In [34]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["mob"], num_docs=second.shape[0])
mob_df=pd.DataFrame({'doc':documents, 'id':document_ids, 'score':document_scores})
mob_df.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-mob-topic.csv')

In [35]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["wizard"], num_docs=second.shape[0])
wizard_df=pd.DataFrame({'doc':documents, 'id':document_ids, 'score':document_scores})
wizard_df.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-wizard-topic.csv')